based on https://www.kaggle.com/isaienkov/keras-nn-with-embeddings-for-cat-features-1-15

(this is a keras tensorflow so no need to change /.keras/keras.json)

# Dense NN 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

./sample_submission.csv
./submission_withoutleak001.csv.gz
./submission_nn007lofo.csv.gz
./generate_datasets.ipynb
./rows_to_drop.csv
./building_all_meters.feather
./train_with_cnt_per_tmp_meter.feather
./test_simple_cleanup.feather
./site4.csv
./winning sub.ipynb
./level2--ensembling_model.ipynb
./untitled.txt
./submission_multimeter003.csv.gz
./tests_building.ipynb
./level1--submission_multimeter003.ipynb
./site1.pkl
./level1--submission_whatsyourcv3_0052_trncl.ipynb
./weather_train.csv
./generate_leak_data.ipynb
./leak012345_001.feather
./site15_leakage.csv
./train.csv
./submission_multimeter004_nobuild.csv.gz
./test.csv
./weather_test.csv
./submission_ucf_replaced.csv
./level1--submission_nn007lofo--CNN.ipynb
./level1--submission_nn001-DenseNN.ipynb
./submission_whatsyourcv3_0052_trncl.csv.gz
./building_metadata.csv
./asu_2016-2018.csv.zip
./train_cleanup_001.feather
./train_simple_cleanup.feather
./level1--submission_multimeter004_nobuild.ipynb
./level1--submission_withoutleak001.

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tenso

In [2]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df


In [3]:
building_df = pd.read_csv("./building_metadata.csv")
weather_train = pd.read_csv("./weather_train.csv")
train = pd.read_feather("./train_cleanup_001.feather")

train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"])
del weather_train

In [4]:

train.loc[(train['meter']==0) & (train['site_id']==0) & (train['timestamp']<'2016-05-21 00:00:00'), 'drop'] = True
train = train[train['drop']!=True]


In [5]:
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["hour"] = train["timestamp"].dt.hour
train["weekday"] = train["timestamp"].dt.weekday

train = average_imputation(train, 'wind_speed')

beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    train.loc[(train['wind_speed']>=item[1]) & (train['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

del train["timestamp"]

In [6]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            meancol = df[col].mean()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(meancol,inplace=True)  
                
                print('change for', meancol)
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday",  "meter"]

drop_cols = ["sea_level_pressure", "wind_speed", "wind_direction"]

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature", "precip_depth_1_hr", "floor_count", 'beaufort_scale']

feat_cols = categoricals + numericals

In [8]:
target = np.log1p(train["meter_reading"])

del train["meter_reading"] 

train = train.drop(drop_cols, axis = 1)

In [9]:
train, NAlist = reduce_mem_usage(train)

Memory usage of properties dataframe is : 3033.1971435546875  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this col:  875000
dtype after:  uint32
******************************
******************************
Column:  year_built
dtype before:  float64
min for this col:  1900.0
max for this col:  2017.0
change for 1967.0896632561887
dtype after:  float32
******************************
******************************
Column:  floor_count
dtype b

In [10]:
def model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001):

    #Inputs
    site_id = Input(shape=[1], name="site_id")
    building_id = Input(shape=[1], name="building_id")
    meter = Input(shape=[1], name="meter")
    primary_use = Input(shape=[1], name="primary_use")
    square_feet = Input(shape=[1], name="square_feet")
    year_built = Input(shape=[1], name="year_built")
    air_temperature = Input(shape=[1], name="air_temperature")
    cloud_coverage = Input(shape=[1], name="cloud_coverage")
    dew_temperature = Input(shape=[1], name="dew_temperature")
    hour = Input(shape=[1], name="hour")
    precip = Input(shape=[1], name="precip_depth_1_hr")
    weekday = Input(shape=[1], name="weekday")
    beaufort_scale = Input(shape=[1], name="beaufort_scale")
   
    #Embeddings layers
    emb_site_id = Embedding(16, 2)(site_id)
    emb_building_id = Embedding(1449, 6)(building_id)
    emb_meter = Embedding(4, 2)(meter)
    emb_primary_use = Embedding(16, 2)(primary_use)
    emb_hour = Embedding(24, 3)(hour)
    emb_weekday = Embedding(7, 2)(weekday)

    concat_emb = concatenate([
           Flatten() (emb_site_id)
         , Flatten() (emb_building_id)
         , Flatten() (emb_meter)
         , Flatten() (emb_primary_use)
         , Flatten() (emb_hour)
         , Flatten() (emb_weekday)
    ])
    
    categ = Dropout(dropout1)(Dense(dense_dim_1,activation='relu') (concat_emb))
    categ = BatchNormalization()(categ)
    categ = Dropout(dropout2)(Dense(dense_dim_2,activation='relu') (categ))
    
    #main layer
    main_l = concatenate([
          categ
        , square_feet
        , year_built
        , air_temperature
        , cloud_coverage
        , dew_temperature
        , precip
        , beaufort_scale
    ])
    
    main_l = Dropout(dropout3)(Dense(dense_dim_3,activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    
    main_2 = concatenate([
          main_l
        , Flatten() (emb_building_id)
        , Flatten() (emb_site_id)

    ])
    
    main_2 = Dropout(dropout4)(Dense(dense_dim_4,activation='relu') (main_2))
    
    #output
    output = Dense(1) (main_2)

    model = Model([ site_id,
                    building_id, 
                    meter, 
                    primary_use, 
                    square_feet, 
                    year_built, 
                   
                    air_temperature,
                    cloud_coverage,
                    dew_temperature, 
                    hour,
                    weekday, 
                    precip,
                    beaufort_scale], output)

    model.compile(optimizer = Adam(lr=lr),
                  loss= mse_loss,
                  metrics=[root_mean_squared_error])
    return model

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [11]:
def get_keras_data(df, num_cols, cat_cols):
    cols = cat_cols + num_cols
    X = {col: np.array(df[col]) for col in cols}
    return X

def train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold, patience=3):
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    model_checkpoint = ModelCheckpoint("model_" + str(fold) + ".hdf5",
                                       save_best_only=True, verbose=1, monitor='val_root_mean_squared_error', mode='min')

    hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=1,
                            callbacks=[early_stopping, model_checkpoint])

    keras_model = load_model("model_" + str(fold) + ".hdf5", custom_objects={'root_mean_squared_error': root_mean_squared_error})
    
    return keras_model

In [15]:
from sklearn.model_selection import KFold, StratifiedKFold

oof = np.zeros(len(train))
batch_size = 256
epochs = 15
models = []

folds = 2
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, train['building_id'])):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_t = get_keras_data(X_train, numericals, categoricals)
    X_v = get_keras_data(X_valid, numericals, categoricals)
    
    keras_model = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    print('*'* 50)

Fold: 0
Train on 9802217 samples, validate on 9802854 samples
Epoch 1/15
9802217/9802217 [==============================] - 263s 27us/step - loss: 1.4013 - root_mean_squared_error: 1.1675 - val_loss: 1.4058 - val_root_mean_squared_error: 1.1214

Epoch 00001: val_root_mean_squared_error improved from inf to 1.12139, saving model to model_0.hdf5
Epoch 2/15
9802217/9802217 [==============================] - 267s 27us/step - loss: 0.9790 - root_mean_squared_error: 0.9833 - val_loss: 1.1196 - val_root_mean_squared_error: 1.0104

Epoch 00002: val_root_mean_squared_error improved from 1.12139 to 1.01044, saving model to model_0.hdf5
Epoch 3/15
9802217/9802217 [==============================] - 263s 27us/step - loss: 0.8935 - root_mean_squared_error: 0.9391 - val_loss: 1.0861 - val_root_mean_squared_error: 0.9947

Epoch 00003: val_root_mean_squared_error improved from 1.01044 to 0.99470, saving model to model_0.hdf5
Epoch 4/15
9802217/9802217 [==============================] - 263s 27us/step -

9802854/9802854 [==============================] - 531s 54us/step - loss: 0.7833 - root_mean_squared_error: 0.8793 - val_loss: 1.0848 - val_root_mean_squared_error: 0.9926

Epoch 00015: val_root_mean_squared_error did not improve from 0.96794
**************************************************


In [25]:
from sklearn.model_selection import KFold, StratifiedKFold

oof = np.zeros(len(train))
batch_size = 256
epochs = 15
models = []

folds = 2
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train, train['building_id'])):
    print('Fold:', fold_n)
    X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_t = get_keras_data(X_train, numericals, categoricals)
    X_v = get_keras_data(X_valid, numericals, categoricals)
    
    keras_model = model(dense_dim_1=64, dense_dim_2=32, dense_dim_3=32, dense_dim_4=16, 
                        dropout1=0.2, dropout2=0.1, dropout3=0.1, dropout4=0.1, lr=0.001)
    mod = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, fold_n, patience=3)
    models.append(mod)
    print('*'* 50)
    

Fold: 0
Train on 9802217 samples, validate on 9802854 samples
Epoch 1/15
9802217/9802217 [==============================] - 169s 17us/step - loss: 1.4258 - root_mean_squared_error: 1.1737 - val_loss: 1.3367 - val_root_mean_squared_error: 1.0953

Epoch 00001: val_root_mean_squared_error improved from inf to 1.09530, saving model to model_0.hdf5
Epoch 2/15
9802217/9802217 [==============================] - 169s 17us/step - loss: 0.9632 - root_mean_squared_error: 0.9753 - val_loss: 1.1054 - val_root_mean_squared_error: 1.0052

Epoch 00002: val_root_mean_squared_error improved from 1.09530 to 1.00519, saving model to model_0.hdf5
Epoch 3/15
9802217/9802217 [==============================] - 167s 17us/step - loss: 0.9015 - root_mean_squared_error: 0.9434 - val_loss: 1.1049 - val_root_mean_squared_error: 1.0038

Epoch 00003: val_root_mean_squared_error improved from 1.00519 to 1.00382, saving model to model_0.hdf5
Epoch 4/15
9802217/9802217 [==============================] - 169s 17us/step -

In [16]:
numericals

['square_feet',
 'year_built',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'floor_count',
 'beaufort_scale']

In [18]:
import gc
del train, target, X_train, X_valid, y_train, y_valid, X_t, X_v, kf
gc.collect()

5

In [19]:
test = pd.read_csv("./test.csv")
test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
del building_df
gc.collect()
test["primary_use"] = le.transform(test["primary_use"])

weather_test = pd.read_csv("./weather_test.csv")

test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_test

In [20]:
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = test["timestamp"].dt.hour
test["weekday"] = test["timestamp"].dt.weekday

test = average_imputation(test, 'wind_speed')

for item in beaufort:
    test.loc[(test['wind_speed']>=item[1]) & (test['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

    
test = test[feat_cols]
test, NAlist = reduce_mem_usage(test)

Memory usage of properties dataframe is : 6051.91162109375  MB
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
min for this col:  0
max for this col:  23
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
min for this col:  0
max for this col:  6
dtype after:  uint8
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype 

In [21]:
from tqdm import tqdm
i=0
res = np.zeros((test.shape[0]),dtype=np.float32)
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/step_size)))):
    for_prediction = get_keras_data(test.iloc[i:i+step_size], numericals, categoricals)
    res[i:min(i+step_size,test.shape[0])] = \
       np.expm1(sum([model.predict(for_prediction, batch_size=1024)[:,0] for model in models])/folds)
    i+=step_size

100%|██████████| 834/834 [09:58<00:00,  1.42it/s]


In [22]:
submission = pd.read_csv('sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_nn001.csv', index=False)
submission

,row_id,meter_reading
0,0,197.040619
1,1,96.668510
2,2,9.802197
3,3,313.558533
4,4,1295.750488
...,...,...
41697595,41697595,7.001263
41697596,41697596,4.312058
41697597,41697597,4.615134
41697598,41697598,172.538712


In [24]:
!kaggle competitions submit -c ashrae-energy-prediction -f submission_nn001.csv -m "Dense NN"

100%|████████████████████████████████████████| 732M/732M [00:27<00:00, 27.8MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III

In [33]:
1+1

2